<a href="https://colab.research.google.com/github/iChutima/iChutima/blob/main/Lunvia_Dubbing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 🎬 LUNVIA DUBBING CINEMA - by Adnan @adroadn instagram
#@markdown ### Settings
target_language = "Urdu" #@param ["Hindi", "Urdu", "English", "Spanish", "French", "German", "Arabic", "Chinese", "Japanese", "Korean", "Russian", "Portuguese", "Bengali", "Tamil", "Telugu"]
voice_type = "Female" #@param ["Female", "Male"]
keep_background = True #@param {type:"boolean"}
voice_boost = 15 #@param {type:"slider", min:0, max:20, step:1}

print("🎬 LUNVIA DUBBING CINEMA")
print("═" * 60)

# ═══════════════════════════════════════════════════════════════════
# INSTALL
# ═══════════════════════════════════════════════════════════════════
print("\n📦 Installing...")
!pip install -q openai-whisper deep-translator edge-tts pydub nest_asyncio 2>/dev/null
!apt-get install -y ffmpeg > /dev/null 2>&1
print("✅ Done!")

# ═══════════════════════════════════════════════════════════════════
# IMPORTS & SETUP
# ═══════════════════════════════════════════════════════════════════
print("\n📚 Loading...")

import os
import subprocess
import warnings
import torch
import whisper
import asyncio
import nest_asyncio
import edge_tts
from deep_translator import GoogleTranslator
from pydub import AudioSegment

# FIX ASYNCIO IN COLAB
nest_asyncio.apply()

warnings.filterwarnings("ignore")

for d in ["/content/uploads", "/content/outputs", "/content/temp"]:
    os.makedirs(d, exist_ok=True)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"   ✅ Device: {DEVICE}")

# ═══════════════════════════════════════════════════════════════════
# LOAD WHISPER
# ═══════════════════════════════════════════════════════════════════
print("\n📝 Loading Whisper...")
whisper_model = whisper.load_model("base", device=DEVICE)
print("   ✅ Whisper ready!")

# ═══════════════════════════════════════════════════════════════════
# CONFIG
# ═══════════════════════════════════════════════════════════════════
LANG_CODES = {
    "Hindi": "hi", "Urdu": "ur", "English": "en", "Spanish": "es",
    "French": "fr", "German": "de", "Arabic": "ar", "Chinese": "zh-CN",
    "Japanese": "ja", "Korean": "ko", "Russian": "ru", "Portuguese": "pt",
    "Bengali": "bn", "Tamil": "ta", "Telugu": "te"
}

VOICES = {
    "Hindi": {"Female": "hi-IN-SwaraNeural", "Male": "hi-IN-MadhurNeural"},
    "Urdu": {"Female": "ur-PK-UzmaNeural", "Male": "ur-PK-AsadNeural"},
    "English": {"Female": "en-US-JennyNeural", "Male": "en-US-GuyNeural"},
    "Spanish": {"Female": "es-ES-ElviraNeural", "Male": "es-ES-AlvaroNeural"},
    "French": {"Female": "fr-FR-DeniseNeural", "Male": "fr-FR-HenriNeural"},
    "German": {"Female": "de-DE-KatjaNeural", "Male": "de-DE-ConradNeural"},
    "Arabic": {"Female": "ar-SA-ZariyahNeural", "Male": "ar-SA-HamedNeural"},
    "Chinese": {"Female": "zh-CN-XiaoxiaoNeural", "Male": "zh-CN-YunxiNeural"},
    "Japanese": {"Female": "ja-JP-NanamiNeural", "Male": "ja-JP-KeitaNeural"},
    "Korean": {"Female": "ko-KR-SunHiNeural", "Male": "ko-KR-InJoonNeural"},
    "Russian": {"Female": "ru-RU-SvetlanaNeural", "Male": "ru-RU-DmitryNeural"},
    "Portuguese": {"Female": "pt-BR-FranciscaNeural", "Male": "pt-BR-AntonioNeural"},
    "Bengali": {"Female": "bn-IN-TanishaaNeural", "Male": "bn-IN-BashkarNeural"},
    "Tamil": {"Female": "ta-IN-PallaviNeural", "Male": "ta-IN-ValluvarNeural"},
    "Telugu": {"Female": "te-IN-ShrutiNeural", "Male": "te-IN-MohanNeural"},
}

# Multi-speaker alternate voices
SPEAKER_VOICES = {
    "Female": ["en-US-JennyNeural", "en-US-AriaNeural", "en-GB-SoniaNeural"],
    "Male": ["en-US-GuyNeural", "en-US-ChristopherNeural", "en-GB-RyanNeural"]
}

# ═══════════════════════════════════════════════════════════════════
# TTS FUNCTION (FIXED!)
# ═══════════════════════════════════════════════════════════════════
async def generate_tts(text, voice_id, output_path):
    """Generate TTS using edge-tts async (with nest_asyncio fix)"""
    try:
        communicate = edge_tts.Communicate(text, voice_id)
        await communicate.save(output_path)
        return True
    except Exception as e:
        print(f"      TTS Error: {e}")
        return False

def synthesize(text, voice_id, output_path):
    """Wrapper to run async TTS"""
    loop = asyncio.get_event_loop()
    return loop.run_until_complete(generate_tts(text, voice_id, output_path))

def format_time(sec):
    h, m, s = int(sec//3600), int((sec%3600)//60), int(sec%60)
    ms = int((sec%1)*1000)
    return f"{h:02d}:{m:02d}:{s:02d},{ms:03d}"

# ═══════════════════════════════════════════════════════════════════
# MAIN
# ═══════════════════════════════════════════════════════════════════
print("\n" + "═" * 60)
print("✅ READY!")
print("═" * 60)
print(f"\n🎯 Language: {target_language} | Voice: {voice_type}")
print("\n📤 Upload your video...")

from google.colab import files
uploaded = files.upload()

if not uploaded:
    print("❌ No file!")
else:
    filename = list(uploaded.keys())[0]
    video_path = f"/content/uploads/{filename}"
    with open(video_path, 'wb') as f:
        f.write(uploaded[filename])

    print(f"\n✅ Uploaded: {filename}")
    print("\n" + "═" * 60)
    print("🎬 STARTING")
    print("═" * 60)

    try:
        # PHASE 1: Extract Audio
        print("\n📂 [1/6] Extracting audio...")
        whisper_audio = "/content/temp/whisper.wav"
        hq_audio = "/content/temp/hq.wav"
        subprocess.run(["ffmpeg", "-i", video_path, "-vn", "-ar", "16000", "-ac", "1", whisper_audio, "-y"], capture_output=True)
        subprocess.run(["ffmpeg", "-i", video_path, "-vn", "-ar", "44100", "-ac", "2", hq_audio, "-y"], capture_output=True)
        original = AudioSegment.from_wav(hq_audio)
        print(f"   ✅ {len(original)/1000:.1f}s extracted")

        # PHASE 2: Transcribe
        print("\n📝 [2/6] Transcribing...")
        result = whisper_model.transcribe(whisper_audio, verbose=False)
        segments = []
        for i, seg in enumerate(result.get("segments", [])):
            text = seg["text"].strip()
            if text:
                segments.append({
                    "start": seg["start"],
                    "end": seg["end"],
                    "text": text,
                    "speaker": i % 2  # Alternate speakers
                })
        print(f"   ✅ {len(segments)} segments found")
        print(f"   ✅ Language: {result.get('language', 'unknown')}")

        if len(segments) == 0:
            print("❌ No speech detected!")
        else:
            # PHASE 3: Translate
            print(f"\n🌍 [3/6] Translating to {target_language}...")
            target_code = LANG_CODES.get(target_language, "hi")
            for i, seg in enumerate(segments):
                try:
                    seg["translated"] = GoogleTranslator(source='auto', target=target_code).translate(seg["text"])
                except:
                    seg["translated"] = seg["text"]
                print(f"   [{i+1}/{len(segments)}] \"{seg['text'][:20]}...\" → \"{seg['translated'][:20]}...\"")
            print("   ✅ Translation done!")

            # PHASE 4: Voice Synthesis
            print(f"\n🎙️ [4/6] Generating voices...")
            base_voice = VOICES.get(target_language, VOICES["Hindi"]).get(voice_type, "hi-IN-SwaraNeural")
            print(f"   Base voice: {base_voice}")

            synth_data = []
            for i, seg in enumerate(segments):
                mp3_path = f"/content/temp/v_{i}.mp3"

                # Use base voice for target language
                voice_id = base_voice

                success = synthesize(seg["translated"], voice_id, mp3_path)

                if success and os.path.exists(mp3_path) and os.path.getsize(mp3_path) > 100:
                    try:
                        audio = AudioSegment.from_mp3(mp3_path)
                        audio = audio + voice_boost  # Volume boost
                        synth_data.append({
                            "audio": audio,
                            "start_ms": int(seg["start"] * 1000),
                            "speaker": seg["speaker"]
                        })
                        print(f"   ✅ [{i+1}/{len(segments)}] {len(audio)}ms generated")
                    except Exception as e:
                        print(f"   ⚠️ [{i+1}/{len(segments)}] Load error: {e}")
                else:
                    print(f"   ⚠️ [{i+1}/{len(segments)}] TTS failed")

            print(f"\n   ✅ Generated: {len(synth_data)}/{len(segments)}")

            if len(synth_data) == 0:
                print("❌ No voices generated! Check internet connection.")
            else:
                # PHASE 5: Mix Audio
                print("\n🎚️ [5/6] Mixing audio...")
                if keep_background:
                    bg = original - 22  # Duck background
                    print("   📉 Background at -22dB")
                else:
                    bg = AudioSegment.silent(duration=len(original))
                    print("   🔇 Silent background")

                final = bg
                for i, s in enumerate(synth_data):
                    pos = s["start_ms"]
                    if pos < len(final):
                        final = final.overlay(s["audio"], position=pos)
                        print(f"   🔊 Voice {i+1} at {pos/1000:.1f}s")

                final_path = "/content/outputs/dubbed_audio.wav"
                final.export(final_path, format="wav")
                print(f"   ✅ Mixed: {len(final)/1000:.1f}s")

                # PHASE 6: Create Video
                print("\n🎬 [6/6] Creating video...")
                output_video = f"/content/outputs/Lunvia_{target_language}.mp4"
                subprocess.run([
                    "ffmpeg", "-i", video_path, "-i", final_path,
                    "-c:v", "copy", "-map", "0:v:0", "-map", "1:a:0",
                    "-shortest", output_video, "-y"
                ], capture_output=True)
                print("   ✅ Video created!")

                # SRT
                srt = ""
                for i, seg in enumerate(segments):
                    srt += f"{i+1}\n{format_time(seg['start'])} --> {format_time(seg['end'])}\n{seg['translated']}\n\n"
                srt_path = "/content/outputs/subtitles.srt"
                with open(srt_path, "w", encoding="utf-8") as f:
                    f.write(srt)

                # Script
                script = f"LUNVIA DUBBING - {target_language}\n{'='*40}\n\n"
                for seg in segments:
                    script += f"[{seg['start']:.1f}s] {seg['text']}\n→ {seg['translated']}\n\n"
                script_path = "/content/outputs/script.txt"
                with open(script_path, "w", encoding="utf-8") as f:
                    f.write(script)

                # SUCCESS
                print("\n" + "═" * 60)
                print("🎉 COMPLETE!")
                print("═" * 60)

                if os.path.exists(output_video):
                    size = os.path.getsize(output_video) / (1024*1024)
                    print(f"\n📊 Output: {size:.1f} MB")

                    print("\n📥 Downloading...")
                    files.download(output_video)
                    files.download(srt_path)
                    files.download(script_path)

                    print("\n✅ FILES:")
                    print(f"   📹 Lunvia_{target_language}.mp4")
                    print("   📄 subtitles.srt")
                    print("   📝 script.txt")
                else:
                    print("❌ Video creation failed!")

    except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()


🎬 LUNVIA DUBBING CINEMA
════════════════════════════════════════════════════════════

📦 Installing...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 17.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.3 MB/s eta 0:00:00
✅ Done!

📚 Loading...


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


   ✅ Device: cuda

📝 Loading Whisper...


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 265MiB/s]


   ✅ Whisper ready!

════════════════════════════════════════════════════════════
✅ READY!
════════════════════════════════════════════════════════════

🎯 Language: Urdu | Voice: Female

📤 Upload your video...


Saving Fantasy Becomes Real_EP001.mp4 to Fantasy Becomes Real_EP001.mp4

✅ Uploaded: Fantasy Becomes Real_EP001.mp4

════════════════════════════════════════════════════════════
🎬 STARTING
════════════════════════════════════════════════════════════

📂 [1/6] Extracting audio...
   ✅ 175.4s extracted

📝 [2/6] Transcribing...
Detected language: Chinese


100%|██████████| 17540/17540 [00:12<00:00, 1452.61frames/s]


   ✅ 85 segments found
   ✅ Language: zh

🌍 [3/6] Translating to Urdu...
   [1/85] "好可愛..." → "好可愛..."
   [2/85] "太可愛..." → "太可愛..."
   [3/85] "小小..." → "چھوٹا..."
   [4/85] "我帶人家..." → "我帶人家..."
   [5/85] "給你來相親來了..." → "給你來相親來了..."
   [6/85] "姑娘們..." → "姑娘們..."
   [7/85] "快進來..." → "快進來..."
   [8/85] "鰍的..." → "鰍的..."
   [9/85] "孩子..." → "بچہ..."
   [10/85] "你們兩個..." → "你們兩個..."
   [11/85] "都是很有じ..." → "وہ سب بہت اچھے ہیں۔..."
   [12/85] "太可愛了..." → "太可愛了..."
   [13/85] "了會不會高..." → "了會不會高..."
   [14/85] "還會一直..." → "還會一直..."
   [15/85] "这个可以..." → "یہ کیا جا سکتا ہے۔..."
   [16/85] "好..." → "اچھا..."
   [17/85] "周哥哥嫌我..." → "بھائی چاؤ مجھے ناپسن..."
   [18/85] "我讲二百段彩蛋..." → "میں آپ کو دو سو ایسٹ..."
   [19/85] "周哥哥..." → "چاؤ بھائی..."
   [20/85] "我只要一百半彩礼..." → "مجھے صرف ڈیڑھ سو جہی..."
   [21/85] "也我要跟你生猴子..." → "میں بھی تمہارے ساتھ ..."
   [22/85] "周哥哥..." → "چاؤ بھائی..."
   [23/85] "行吧..." → "ٹھیک ہے..."
   [24/85] "我们要在一起..." → "ہم ایک ساتھ رہنا چاہ..."
   [25/85] "死化他们粘..." 